# Crown

This notebook that analysis the Crown.

---------------

##### Imports

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import os
import math
import time
import trimesh
import shapely
import pymeshfix
import subprocess
import alphashape
import numpy as np
import open3d as o3d
import logging as log
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial import KDTree
from descartes import PolygonPatch
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from plyfile import PlyData, PlyElement
from skimage.measure import CircleModel, ransac
from sklearn.neighbors import NearestNeighbors
import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.o3d_utils as o3d_utils
from misc.quaternion import Quaternion

In [ ]:
LEAF_GREEN = [0,0.78,0]

class Crown:
    """ This class implements the AdTree delft repository.
    Attributes:
        ---
    """

    def __init__(self):
        pass

    def pcd_convex_hull(sefl, pcd, plot=True):
        log.info(f'Convex Hull for pcd with {len(pcd.points)} points.')
        o3d_mesh, _ = pcd.compute_convex_hull()
        o3d_mesh.paint_uniform_color(LEAF_GREEN)
        o3d_mesh.compute_vertex_normals()
        log.info(f'Done. Crown volume: {o3d_mesh.get_volume():.2f}m3')
        
        if plot: # Visualize
            plot_mesh(o3d_mesh)

        return o3d_mesh

    def alpha_shape(self, pcd, alpha=.8, plot=True):
        log.info(f'Alpha Shapes for pcd with {len(pcd.points)} points.')
        start = time.time()
        pcd_pts = np.asarray(pcd.points)
        mesh = alphashape.alphashape(pcd_pts, alpha)
        log.info(f'Done. {time.time()-start:.2f}s.')
        
        # Repair
        log.info(f'Repair broken faces...')
        clean_points, clean_faces = pymeshfix.clean_from_arrays(mesh.vertices,  mesh.faces)
        mesh = trimesh.base.Trimesh(clean_points, clean_faces)
        mesh.fix_normals()
        log.info(f'Done. Crown volume: {mesh.volume:.2f}m3')
        
        # convert
        o3d_mesh = mesh.as_open3d
        o3d_mesh.compute_vertex_normals()
        o3d_mesh.paint_uniform_color(LEAF_GREEN)

        if plot: # Visualize
            plot_mesh(o3d_mesh)

        return o3d_mesh

    def model(self, pcd):
        pass


def plot_mesh(mesh):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_trisurf(*zip(*mesh.vertices), triangles=mesh.triangles, color='green')
    plt.show()

def show_meshlines(mesh, pcds=[]):
    mesh_lines = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
    mesh_lines.paint_uniform_color((1, 0, 0))
    geometries = pcds + [mesh_lines]
    o3d.visualization.draw_geometries(geometries)


##### Load data

In [ ]:
# Load point cloud data
source = 'x'

if source == 'ahn':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_AHN.las')
elif source == 'cyclo':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Cyclo.las')
else:
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Sonarski.las')


stem_cloud = pcd.select_by_index(np.where(np.asarray(pcd.points)[:,2]<4.8)[0])
crown_cloud = pcd.select_by_index(np.where(np.asarray(pcd.points)[:,2]>5)[0])

##### Experimental

In [ ]:
crownAnalysis = Crown()

In [ ]:
voxel_size = 0.3

# Down sample crown
crown_sampled = crown_cloud.voxel_down_sample(voxel_size)

In [ ]:
crownAnalysis.model(crown_cloud)

In [ ]:
mesh_ch = crownAnalysis.pcd_convex_hull(crown_sampled)
mesh_as = crownAnalysis.alpha_shape(crown_sampled)

In [ ]:
show_meshlines(mesh_as, [crown_cloud])

In [ ]:
def grid_project(pcd, voxel_size):
    pts = np.array(pcd.points)
    pts[:,2] = 0
    pcd_ = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pts))
    pcd_ = pcd_.voxel_down_sample(voxel_size)
    pts = np.asarray(pcd_.points)[:,:2]
    return pts

In [ ]:
# Stem shape
proj_pts = grid_project(stem_cloud, 0.02)
stem_shape = alphashape.alphashape(proj_pts, 0.5)
stem_shape = stem_shape.buffer(0.01)
tree_center = np.hstack(stem_shape.centroid.coords)

# Crown shape
proj_pts = grid_project(crown_cloud, 0.3)
crown_shape = alphashape.alphashape(proj_pts, 2)
crown_shape = crown_shape.buffer(0.1)

# Center
proj_pts_shifted = proj_pts-[tree_center]
stem_shape_shifted = shapely.affinity.translate(stem_shape, *(-tree_center))
crown_shape_shifted = shapely.affinity.translate(crown_shape, *(-tree_center))

# Initialize plot
fig, ax = plt.subplots()
ax.plot(0, marker='x', c='k', markersize=5)
ax.add_patch(PolygonPatch(crown_shape_shifted, alpha=.15, color='green', label='Crown'))
ax.add_patch(PolygonPatch(stem_shape_shifted, alpha=.8, color='brown', label='Stem'))
lim = np.max(np.abs(proj_pts_shifted))
ax.set_xlim(-lim-.5,lim+.5)
ax.set_ylim(-lim-.5,lim+.5)
ax.legend()
ax.set_title('Tree Projection')
plt.show()